In [1]:
from keras.applications.mobilenet_v2 import MobileNetV2 
from keras.applications.mobilenet import MobileNet
from keras.applications.densenet import DenseNet169
from keras.applications.resnet50 import ResNet50
from keras.applications.nasnet import NASNetMobile, NASNetLarge
from keras.applications.xception import Xception
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Dense, Dropout
from keras.optimizers import Adam, SGD, Adagrad, Adadelta
from keras import backend as K
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import imutils
import random
import pickle
import cv2
import os
import math

%matplotlib inline

Using TensorFlow backend.


In [2]:
# DATASET_PATH = r"G:\landmarks_of_melbourne_sml"
DATASET_PATH = r"E:\dataset15"
TESTSET_PATH = r"E:\testset15"

# initialize the number of epochs to train for, initial learning rate, batch size, and image dimensions
EPOCHS = 15
INIT_LR = 0.001
BS = 8
LABEL_NUM = 15
READING_BS = 1000

# image dimension may need to change according to the model spec
IMAGE_DIMS =  (224, 224, 3)
IMAGE_DIMS =  (299, 299, 3)

# initialize the data and labels
data = []
labels = []

data_test =[]
labels_test=[]

# grab the image paths and randomly shuffle them
imagePaths = sorted(list(paths.list_images(DATASET_PATH)))
random.Random(881278).shuffle(imagePaths)

testPaths = sorted(list(paths.list_images(TESTSET_PATH)))

In [3]:
# loop over the test images
for testPath in testPaths:
    # load the image, pre-process it, and store it in the data_test list
    assert os.path.isfile(testPath)
    image = cv2.imread(testPath)
    image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
    image = img_to_array(image)
    data_test.append(image)

    # extract the class label from the image path and update the labels_test list
    label = testPath.split(os.path.sep)[-2]
    labels_test.append(label)
    
# scale the raw pixel intensities to the range [0, 1]
data_test = np.array(data_test, dtype="float") / 255.0
labels_test = np.array(labels_test)
print("test data matrix: {:.2f}MB".format(data_test.nbytes / (1024 * 1000.0)))

# binarize the labels
lb = LabelBinarizer()
labels_test = lb.fit_transform(labels_test)

test data matrix: 1565.22MB


In [4]:
# create a custom input layer that accepts images of shape IMAGE_DIMS
input_tensor = Input(shape=IMAGE_DIMS)

# grab the Keras built-in model for transfer learning
base_model = Xception(
        include_top=False,
        weights='imagenet',
        input_tensor=input_tensor,
        input_shape=IMAGE_DIMS,
        pooling='avg')

# original model is trained on different dataset with different number of classes
# so we need to append our own output layer
output_tensor = Dense(LABEL_NUM, activation='softmax')(base_model.output)

model = Model(inputs=input_tensor, outputs=output_tensor)

model.compile(Adam(lr=1e-4), loss="categorical_crossentropy", metrics=['accuracy', 'top_k_categorical_accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
chunk_paths = [imagePaths[i:i + READING_BS] for i in range(0, len(imagePaths), READING_BS)]
for chunk_path in chunk_paths:
    # loop over the each chuck
    data = []
    labels = []
    for imagePath in chunk_path:
        # load the image, pre-process it, and store it in the data list
        assert os.path.isfile(imagePath)
        image = cv2.imread(imagePath)
        image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
        image = img_to_array(image)
        data.append(image)

        # extract the class label from the image path and update the labels list
        label = imagePath.split(os.path.sep)[-2]
        labels.append(label)
    
    # construct the image generator for data augmentation
    # aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1, height_shift_range=0.1, 
    #                          shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
    aug = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1)
    
    # scale the raw pixel intensities to the range [0, 1]
    data = np.array(data, dtype="float") / 255.0
    labels = np.array(labels)
    print("training data matrix: {:.2f}MB".format(data.nbytes / (1024 * 1000.0)))

    # binarize the labels
    lb = LabelBinarizer()
    labels = lb.fit_transform(labels)
    
    h = model.fit_generator(
        aug.flow(data, labels, batch_size=BS),
        validation_data=(data_test, labels_test),
        steps_per_epoch=math.ceil(len(data)/BS),
        epochs=EPOCHS, verbose=1)

training data matrix: 2095.34MB
Instructions for updating:
Use tf.cast instead.
Epoch 1/15
125/125 [==============================] - 49s 390ms/step - loss: 1.1865 - acc: 0.7210 - top_k_categorical_accuracy: 0.9010 - val_loss: 0.4302 - val_acc: 0.8996 - val_top_k_categorical_accuracy: 0.9853
Epoch 2/15
125/125 [==============================] - 39s 309ms/step - loss: 0.2177 - acc: 0.9590 - top_k_categorical_accuracy: 0.9960 - val_loss: 0.1328 - val_acc: 0.9772 - val_top_k_categorical_accuracy: 0.9906
Epoch 3/15
125/125 [==============================] - 39s 311ms/step - loss: 0.0623 - acc: 0.9900 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.0888 - val_acc: 0.9759 - val_top_k_categorical_accuracy: 0.9920
Epoch 4/15
125/125 [==============================] - 39s 315ms/step - loss: 0.0508 - acc: 0.9900 - top_k_categorical_accuracy: 0.9990 - val_loss: 0.0957 - val_acc: 0.9746 - val_top_k_categorical_accuracy: 0.9946
Epoch 5/15
125/125 [==============================] - 39s 315ms/step

125/125 [==============================] - 38s 306ms/step - loss: 0.0048 - acc: 0.9990 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.0657 - val_acc: 0.9880 - val_top_k_categorical_accuracy: 0.9987
Epoch 8/15
125/125 [==============================] - 38s 306ms/step - loss: 0.0025 - acc: 1.0000 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.0635 - val_acc: 0.9866 - val_top_k_categorical_accuracy: 0.9987
Epoch 9/15
125/125 [==============================] - 38s 306ms/step - loss: 0.0013 - acc: 1.0000 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.0650 - val_acc: 0.9866 - val_top_k_categorical_accuracy: 0.9987
Epoch 10/15
125/125 [==============================] - 38s 306ms/step - loss: 0.0022 - acc: 0.9990 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.0661 - val_acc: 0.9893 - val_top_k_categorical_accuracy: 0.9987
Epoch 11/15
125/125 [==============================] - 38s 306ms/step - loss: 0.0038 - acc: 1.0000 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.0854 - v

125/125 [==============================] - 38s 307ms/step - loss: 0.0023 - acc: 1.0000 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.0438 - val_acc: 0.9906 - val_top_k_categorical_accuracy: 0.9987
training data matrix: 1569.41MB
Epoch 1/15
94/94 [==============================] - 32s 338ms/step - loss: 0.1730 - acc: 0.9628 - top_k_categorical_accuracy: 0.9960 - val_loss: 0.0781 - val_acc: 0.9759 - val_top_k_categorical_accuracy: 0.9973
Epoch 2/15
94/94 [==============================] - 30s 324ms/step - loss: 0.0638 - acc: 0.9814 - top_k_categorical_accuracy: 0.9960 - val_loss: 0.0185 - val_acc: 0.9893 - val_top_k_categorical_accuracy: 1.0000
Epoch 3/15
94/94 [==============================] - 30s 324ms/step - loss: 0.0195 - acc: 0.9920 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.0217 - val_acc: 0.9920 - val_top_k_categorical_accuracy: 1.0000
Epoch 4/15
94/94 [==============================] - 31s 325ms/step - loss: 0.0048 - acc: 0.9987 - top_k_categorical_accuracy: 1.0000 

In [6]:
# save the model to disk
model.save("E:\models\Xception-dataset15-d299-e15")

In [7]:
# save the label binarizer to disk
f = open(r"E:\models\lb-dataset15-2", "wb")
f.write(pickle.dumps(lb))
f.close()